In [1]:
from pathlib import Path
import json
from functools import reduce
import math
import datetime as dt
import pytz 
from itertools import product
from collections import OrderedDict
import time
import sys

import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.ops as so
from requests_html import HTMLSession
    
import helpers as hp

%load_ext autoreload
%autoreload 2


In [2]:
def get_rent_data_urls():
    """
    Scrape MBIE's website to get the location of three files of rental bond data.
    Return a list of three dictionaries, each with the following keys and values.
    
    - ``'kind'``: the kind of data; one of 'rent_count', 'rent_mean', 'rent_geo_mean'
    - ``'filename'``: the name of the CSV data file
    - ``'url'``: the URL of the CSV data file
    
    """
    src_url = "https://www.mbie.govt.nz/building-and-energy/tenancy-and-housing/rental-bond-data/"

    # Scrape the HTML page above to get the data urls of interest
    session = HTMLSession()
    r = session.get(src_url)
    urls = [
        link for link in r.html.absolute_links 
        if "Quarterly" in link 
        and link.split("/")[-1].startswith("detailed-")
    ]
    
    # Filter urls to finer set and include some metadata
    return [
        {
            "kind": "rent_count",
            "filename": "detailed-lodged-bonds.csv",
            "url": [u for u in urls if u.endswith("detailed-lodged-bonds.csv")][0],
            "target_path": hp.DATA_DIR/'collected'/"detailed-lodged-bonds.csv",
        },
        {
            "kind": "rent_mean",
            "filename": "detailed-mean-rents.csv",
            "url": [u for u in urls if u.endswith("detailed-mean-rents.csv")][0],
            "target_path": hp.DATA_DIR/'collected'/"detailed-mean-rents.csv",
        },
        {
            "kind": "rent_geo_mean",
            "filename": "detailed-geo-mean-rents.csv",
            "url": [u for u in urls if u.endswith("detailed-geo-mean-rents.csv")][0],
            "target_path": hp.DATA_DIR/'collected'/"detailed-geo-mean-rents.csv",
        },
    ]


# Download rent data

In [4]:
data = get_rent_data_urls()
    
for d in data:
    r = requests.get(d['url'])
    if not r.ok:
        print("Failed to get", d['filename'])
    else:
        print("Getting and saving", d['filename'])
        path = d["target_path"]
        with path.open('w') as tgt:
            tgt.write(r.text)


Getting and saving detailed-lodged-bonds.csv
Getting and saving detailed-mean-rents.csv
Getting and saving detailed-geo-mean-rents.csv


# Process rent data

In [5]:
# Reshape and merge all rent data sets

def clean(f, kind):
    f = f.copy()
    f = f.rename(columns={
        'SAU': 'au2001',
        'Property_Type': 'property_type',
        'Bedrooms': 'num_bedrooms'
    })

    # Drop subtotals
    cond = False
    for col in ['au2001', 'property_type', 'num_bedrooms']:
        cond |= f[col].str.contains('total', case=False)

    f = f[~cond].copy()
    
    # Reshape
    id_vars = ['au2001', 'property_type', 'num_bedrooms']
    value_vars = [c for c in f.columns if '-' in c]
    f = pd.melt(f, id_vars=id_vars, value_vars=value_vars,
      var_name='quarter', value_name=kind)
    
    return f

frames = []
for d in data:
    path = d["target_path"]
    print(path)
    f = pd.read_csv(path, dtype={'SAU': str})
    frames.append(clean(f, d["kind"]))
    
f = reduce(lambda x, y: pd.merge(x, y), frames)

# Merge in region data
path = hp.get_path('au2001_csv')
g = pd.read_csv(path, dtype={'au2001': str})
f = f.merge(g)

# Write to file
path = hp.get_path('rents')
f.to_csv(path, index=False)
f[f['rent_count'].notnull()].tail()


/home/araichev/mrcagney_projects/affordability_nz/data/collected/detailed-lodged-bonds.csv
/home/araichev/mrcagney_projects/affordability_nz/data/collected/detailed-mean-rents.csv
/home/araichev/mrcagney_projects/affordability_nz/data/collected/detailed-geo-mean-rents.csv


,au2001,property_type,num_bedrooms,quarter,rent_count,rent_mean,rent_geo_mean,au_name,territory,region,rental_area
1059240,612802,House,3,2009-06-01,5.0,188.0,184.0,Riverton West,Southland District,Southland,Clutha/Southland
1059264,612802,House,3,2011-06-01,6.0,198.0,195.0,Riverton West,Southland District,Southland,Clutha/Southland
1059285,612802,House,3,2013-03-01,6.0,218.0,213.0,Riverton West,Southland District,Southland,Clutha/Southland
1059291,612802,House,3,2013-09-01,6.0,207.0,205.0,Riverton West,Southland District,Southland,Clutha/Southland
1059315,612802,House,3,2015-09-01,5.0,230.0,228.0,Riverton West,Southland District,Southland,Clutha/Southland


In [6]:
# Print latest 2 quarters
hp.get_latest_quarters(2)

['2019-12-01', '2020-03-01']

# Explore rents

In [7]:
rents = hp.get_data('rents')
print(rents['quarter'].unique())
rents.head()


['1993-03-01' '1993-06-01' '1993-09-01' '1993-12-01' '1994-03-01'
 '1994-06-01' '1994-09-01' '1994-12-01' '1995-03-01' '1995-06-01'
 '1995-09-01' '1995-12-01' '1996-03-01' '1996-06-01' '1996-09-01'
 '1996-12-01' '1997-03-01' '1997-06-01' '1997-09-01' '1997-12-01'
 '1998-03-01' '1998-06-01' '1998-09-01' '1998-12-01' '1999-03-01'
 '1999-06-01' '1999-09-01' '1999-12-01' '2000-03-01' '2000-06-01'
 '2000-09-01' '2000-12-01' '2001-03-01' '2001-06-01' '2001-09-01'
 '2001-12-01' '2002-03-01' '2002-06-01' '2002-09-01' '2002-12-01'
 '2003-03-01' '2003-06-01' '2003-09-01' '2003-12-01' '2004-03-01'
 '2004-06-01' '2004-09-01' '2004-12-01' '2005-03-01' '2005-06-01'
 '2005-09-01' '2005-12-01' '2006-03-01' '2006-06-01' '2006-09-01'
 '2006-12-01' '2007-03-01' '2007-06-01' '2007-09-01' '2007-12-01'
 '2008-03-01' '2008-06-01' '2008-09-01' '2008-12-01' '2009-03-01'
 '2009-06-01' '2009-09-01' '2009-12-01' '2010-03-01' '2010-06-01'
 '2010-09-01' '2010-12-01' '2011-03-01' '2011-06-01' '2011-09-01'
 '2011-12-

,au2001,property_type,num_bedrooms,quarter,rent_count,rent_mean,rent_geo_mean,au_name,territory,region,rental_area
0,500100,House,2,1993-03-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
1,500100,House,3,1993-03-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
2,500100,House,2,1993-06-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
3,500100,House,3,1993-06-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
4,500100,House,2,1993-09-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North


In [8]:
# Slice in time and aggregate 
agg_rents = hp.aggregate_rents(rents, '2019-09-01')
agg_rents.head()

/home/araichev/mrcagney_projects/affordability_nz/py/helpers.py:188: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_mean'] = (group['rent_mean']*group['rent_count']).sum()/\


,rental_area,num_bedrooms,territory,region,rent_count,rent_mean,rent_geo_mean
0,Addington,1,Christchurch City,Canterbury,160.0,198.181250,190.191337
1,Addington,2,Christchurch City,Canterbury,57.0,339.964912,323.615908
2,Addington,3,Christchurch City,Canterbury,77.0,412.519481,396.742794
3,Addington,4,Christchurch City,Canterbury,0.0,NaN,NaN
4,Addington,5+,Christchurch City,Canterbury,0.0,NaN,NaN


In [9]:
(
    hp.aggregate_rents(rents, '2018-06-01', groupby_cols=('au2001', 'num_bedrooms'))
    .loc[lambda x: x.region == 'Auckland']
)

/home/araichev/mrcagney_projects/affordability_nz/py/helpers.py:188: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_mean'] = (group['rent_mean']*group['rent_count']).sum()/\


,au2001,num_bedrooms,territory,region,rent_count,rent_mean,rent_geo_mean
289,505300,1,Rodney District,Auckland,0.0,NaN,NaN
290,505300,2,Rodney District,Auckland,15.0,398.000000,395.193857
291,505300,3,Rodney District,Auckland,66.0,446.378788,442.124353
292,505300,4,Rodney District,Auckland,17.0,481.764706,479.135350
293,505300,5+,Rodney District,Auckland,0.0,NaN,NaN
...,...,...,...,...,...,...,...
1983,526701,1,Franklin District,Auckland,0.0,NaN,NaN
1984,526701,2,Franklin District,Auckland,0.0,NaN,NaN
1985,526701,3,Franklin District,Auckland,15.0,499.000000,491.849942
1986,526701,4,Franklin District,Auckland,0.0,NaN,NaN


In [10]:
# What fraction of rental data do we have by num_bedrooms?

def hits(group):
    d = {}
    d['hit_frac'] = group['rent_mean'].count()/group['rent_mean'].shape[0]
    return pd.Series(d)

date = '2018-06-01'
f = hp.aggregate_rents(rents, date, groupby_cols=('au2001', 'num_bedrooms'))
cond = f['region'] == 'Auckland'
print('census area units')
print(f[cond].copy().groupby('num_bedrooms').apply(hits).reset_index())

f = hp.aggregate_rents(rents, date, groupby_cols=('rental_area', 'num_bedrooms'))
cond = f['region'] == 'Auckland'
print('rental area units')
print(f[cond].copy().groupby('num_bedrooms').apply(hits).reset_index())


/home/araichev/mrcagney_projects/affordability_nz/py/helpers.py:188: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_mean'] = (group['rent_mean']*group['rent_count']).sum()/\


census area units
  num_bedrooms  hit_frac
0            1  0.461300
1            2  0.744928
2            3  0.894587
3            4  0.671470
4           5+  0.191824


/home/araichev/mrcagney_projects/affordability_nz/py/helpers.py:188: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_mean'] = (group['rent_mean']*group['rent_count']).sum()/\


rental area units
  num_bedrooms  hit_frac
0            1  0.828283
1            2  1.000000
2            3  1.000000
3            4  0.929293
4           5+  0.393939


# Select latest two quarters and slice into regional chunks

In [11]:
# Get latest two quarters of rents
start_date = hp.get_latest_quarters(2)[0]
rents = hp.get_data('rents')
rents = rents.loc[lambda x: x.quarter >= start_date].copy()

# Create regional slices
for region in hp.REGIONS:
    # Build rents
    region_c = region.capitalize()
    region_rents = rents.loc[lambda x: x.region == region_c].copy()
    print(region, region_rents['quarter'].unique(), '#rows =', region_rents.shape[0])
    path = hp.get_path('rents', region)
    print('  Saving to', path)
    
    # Create region directory if it does not exist
    if not path.parent.exists():
        path.parent.mkdir()
        
    region_rents.to_csv(path, index=False)
    

auckland ['2019-12-01' '2020-03-01'] #rows = 5640
  Saving to /home/araichev/mrcagney_projects/affordability_nz/data/processed/auckland/rents.csv
canterbury ['2019-12-01' '2020-03-01'] #rows = 2574
  Saving to /home/araichev/mrcagney_projects/affordability_nz/data/processed/canterbury/rents.csv
wellington ['2019-12-01' '2020-03-01'] #rows = 2444
  Saving to /home/araichev/mrcagney_projects/affordability_nz/data/processed/wellington/rents.csv


# JSONize regional rents for the web, grouping by rental area and number of bedrooms

In [12]:
for region in hp.REGIONS:
    region_rents = hp.get_data('rents', region)
    d = hp.build_json_rents(region_rents)
    path = hp.get_path('rents_json', region)
    print('**', region, '\n', d)
    with path.open('w') as tgt:
        json.dump(d, tgt)

    

/home/araichev/mrcagney_projects/affordability_nz/py/helpers.py:188: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_mean'] = (group['rent_mean']*group['rent_count']).sum()/\


** auckland 
 {'Albany': {'1': 267.0, '2': 523.0, '3': 633.0, '4': 750.0}, 'Avondale': {'1': 218.0, '2': 459.0, '3': 549.0, '4': 650.0}, 'Balmoral': {'1': 437.0, '2': 503.0, '3': 666.0, '4': 827.0}, 'Beachhaven/Birkdale': {'1': None, '2': 534.0, '3': 564.0, '4': 677.0}, 'Blockhouse Bay/New Windsor': {'1': None, '2': 360.0, '3': 583.0, '4': 668.0}, 'Botony Downs': {'1': None, '2': None, '3': 623.0, '4': 721.0}, 'Browns Bay': {'1': 398.0, '2': 502.0, '3': 612.0, '4': 640.0}, 'Bucklands Beach': {'1': None, '2': None, '3': 668.0, '4': 799.0}, 'Central East': {'1': 356.0, '2': 513.0, '3': 832.0, '4': None}, 'Central West': {'1': 401.0, '2': 522.0, '3': 722.0, '4': None}, 'Chatswood/Birkenhead/Northcote Point': {'1': 429.0, '2': 528.0, '3': 689.0, '4': 808.0}, 'Dannemora': {'1': 451.0, '2': 514.0, '3': 653.0, '4': 731.0}, 'Devonport': {'1': 433.0, '2': 598.0, '3': 735.0, '4': 985.0}, 'East Coast Bays': {'1': None, '2': 592.0, '3': 675.0, '4': 937.0}, 'Eden Terrace': {'1': 385.0, '2': 587.0, 

** wellington 
 {'Brooklyn': {'1': 425.0, '2': 537.0, '3': 690.0, '4': 699.0}, 'Carterton/South Wairarapa': {'1': None, '2': 338.0, '3': 400.0, '4': None}, 'Eastern Bays': {'1': None, '2': 494.0, '3': 666.0, '4': 760.0}, 'Epuni/Avalon': {'1': 186.0, '2': 474.0, '3': 566.0, '4': None}, 'Hataitai': {'1': 369.0, '2': 511.0, '3': 714.0, '4': 831.0}, 'Heretaunga/Silverstream': {'1': None, '2': None, '3': 583.0, '4': 703.0}, 'Hutt Central/Waterloo': {'1': 313.0, '2': 487.0, '3': 584.0, '4': 815.0}, 'Island Bay/Melrose': {'1': 260.0, '2': 539.0, '3': 647.0, '4': 898.0}, 'Johnsonville/Newlands': {'1': 245.0, '2': 484.0, '3': 597.0, '4': 745.0}, 'Karori': {'1': None, '2': 485.0, '3': 694.0, '4': None}, 'Karori South/Makara': {'1': 236.0, '2': 451.0, '3': 654.0, '4': 736.0}, 'Kelburn/Aro Valley': {'1': 366.0, '2': 541.0, '3': 685.0, '4': 936.0}, 'Khandallah': {'1': None, '2': 566.0, '3': 688.0, '4': 879.0}, 'Kilbirnie/Lyall Bay': {'1': 396.0, '2': 500.0, '3': 702.0, '4': None}, 'Kingston/Happy V